In [0]:
# modules
import os, sys
import traceback
import itertools
from functools import reduce
from pyspark.sql import DataFrame
import pyspark
from time import time
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row

from pyspark.sql.functions import col, unix_timestamp, format_number
#import pyspark.sql.functions as F

from pyspark.sql.functions import lit
from datetime import datetime, timedelta
import smtplib
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from pyspark import SparkFiles
import requests
import json
import time

In [0]:
conf = {}

conf['jdbcDriver_origin'] = dbutils.secrets.get('key-vault-company-secrets','sql-company-system-jdbcdriver')
conf['jdbcUsername_origin'] = dbutils.secrets.get('key-vault-company-secrets','sql-company-system-username')
conf['jdbcPassword_origin'] = dbutils.secrets.get('key-vault-company-secrets','sql-company-system-password')
conf['jdbcUrl_origin'] = dbutils.secrets.get('key-vault-company-secrets','sql-company-system-jdbcurl')

conf['jdbcDriver_destination'] = dbutils.secrets.get('key-vault-company-secrets','sql-companyDummy-dw-jdbcdriver')
conf['jdbcUrl_destination'] = dbutils.secrets.get('key-vault-company-secrets','sql-companyDummy-dw-jdbcurl')
conf['jdbcUsername_destination'] = dbutils.secrets.get('key-vault-company-secrets','sql-companyDummy-dw-username')
conf['jdbcPassword_destination'] = dbutils.secrets.get('key-vault-company-secrets','sql-companyDummy-dw-password')

In [0]:
# SQL Azure connection

max_attempts = 15
seconds_to_wait = 5

for attempt in range(max_attempts):

    try:
        dfSQLTestCon = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.Dim_Country").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).load()
        
        print("Connected successfully")
        break
        
    except Exception as e:
        print("Error to connect: {}".format(attempt))
        print(e)
        print("Waiting {} seconds ...".format(seconds_to_wait))
        time.sleep(seconds_to_wait)

Connected successfully


In [0]:
# Dim_Brand: VW_BI_CAT_BRAND

df_dim_brand = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_CAT_BRAND").load()

df_dim_brand.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Brand]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Product: join views VW_CAT_PRODUCT, W_CAT_PRODUCT_TYPE, VW_BI_CAT_TIER, VW_CAT_SEGMENT, VW_BI_CAT_PRODUCT_CATEGORY, VW_BI_SUPER_CATEGORY, VW_CAT_SUB_BRAND

df_dim_product = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""SELECT p.id, p.name, p.active, p.sku, p.ean, p.product_type, pt.name as product_type_name, pt.active as product_type_active, p.volume, p.volume_measure, 
CASE WHEN p.volume_measure = '9LC' THEN 0.11111111 ELSE 1 END as factor, 
p.country_id, p.tier_iid, t.name as tier_name, p.type, 
CASE WHEN p.type = 1 THEN 'Product' WHEN p.type = 2 THEN 'Promotion' end as type_name, 
p.brand_id, p.sub_brand_id, sb.sub_brand_name, p.product_category_id, p.segment_id, s.name as segment_name, s.description as segment_description, p.manufacturer, p.history, p.quantity_per_case, p.primary_UOM, p.secondary_UOM, p.content, p.start_date, p.end_date, image_url  
FROM Staging.VW_CAT_PRODUCT p 
LEFT OUTER JOIN Staging.VW_CAT_TIER t on p.tier_iid = t.id 
LEFT JOIN Staging.VW_CAT_SEGMENT s on p.segment_id = s.id 
LEFT JOIN Staging.VW_CAT_PRODUCT_TYPE pt on p.product_type = pt.id 
LEFT JOIN Staging.VW_CAT_SUB_BRAND sb on p.sub_brand_id = sb.sub_brand_id
""").load()

df_dim_product.createOrReplaceTempView('PRODUCT')

df_product_category = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_CAT_PRODUCT_CATEGORY").load()

df_product_category.createOrReplaceTempView('PRODUCT_CATEGORY')

df_super_category = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""SELECT id, name, active, CASE WHEN include_in_lc_reports=1 THEN 'Y' ELSE 'N' END as super_category_9LC_calculation FROM Staging.VW_BI_SUPER_CATEGORY""").load()

df_super_category.createOrReplaceTempView('SUPER_CATEGORY')

df_dim_all_product = spark.sql("""
SELECT p.id, p.name, p.active, p.sku, p.ean, p.product_type, p.product_type_name, p.product_type_active, p.volume, p.volume_measure, p.factor, p.country_id, p.tier_iid, tier_name, type, type_name, brand_id, sub_brand_id, sub_brand_name, p.segment_id, p.segment_name, p.segment_description, p.manufacturer, p.history, p.quantity_per_case, p.primary_UOM, p.secondary_UOM, p.content, p.start_date, p.end_date, p.product_category_id as category_id, pc.name as category_name, 
CASE WHEN p.product_category_id is not null AND p.volume_measure is not null THEN CONCAT(pc.name,' - ',p.volume_measure) ELSE null END as category_measure,
pc.active as category_acive, pc.super_category_id as super_category_id, sc.name as super_category_name, sc.active as super_category_active, sc.super_category_9LC_calculation, image_url
FROM PRODUCT p 
  LEFT JOIN PRODUCT_CATEGORY pc ON p.product_category_id = pc.id 
  LEFT JOIN SUPER_CATEGORY sc ON pc.super_category_id = sc.id
""")

df_dim_all_product.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Product]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

#display(df_dim_all_product)


In [0]:
# Dim_Product_Chain: VW_PRODUCT_CHAIN

df_dim_product_chain = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_PRODUCT_CHAIN").load()

df_dim_product_chain.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Product_Chain]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_Member (CPG): VW_BI_CAT_MEMBER

df_dim_member = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_CAT_MEMBER").load()

df_dim_member.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Member]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Business: join views VW_BI_BUSINESS_DASHBOARD, VW_CAT_IMAGE_LEVEL, VW_CAT_CHANNEL, VW_CAT_SUB_CHANNEL, VW_BI_CITY, VW_BI_CHAIN_BUSINESS_DASHBOARD

df_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query", """SELECT b.id, b.name, b.channel_id, b.sub_channel_id, b.image_level_id,  b.city_id, b.country_id, b.position_id, b.active,
CASE b.active
  WHEN -1 THEN 'Pending'
  WHEN 0 THEN 'Inactive'
  WHEN 1 THEN 'Active' 
  WHEN 2 THEN 'Rejected'
  WHEN 3 THEN 'Closed' 
  ELSE null 
END as business_status, b.reporting, invitation_code, created_at, updated_at, first_order_date, last_order_date 
FROM Staging.VW_BI_BUSINESS_DASHBOARD b 
""").load()

df_business_active_data = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query", """SELECT *,
case when active = 1 then cast ('9999-12-31' as date)
     when active_new = 1 then eomonth((SELECT max(created_at) FROM companyDummy.Fact_Order f where f.business_id = T1.id )) 
	 end as active_end_date
FROM 
(
select b.id, 
	active, 
	case when active = 1 OR exists (select 1 from Staging.VW_ORDER_DASHBOARD f where b.id = f.business ) then 1 else 0 end as active_new, cast(created_at as Date) as active_start_date
from Staging.VW_BI_BUSINESS_DASHBOARD b
) T1
""").load()

df_image_level = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_CAT_IMAGE_LEVEL").load()

df_channel = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_CAT_CHANNEL").load()

# This view classifies channels into groups according to given definition 

df_channel_group = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT id AS channel_id,
CASE 
	WHEN id IN (2, 9, 10, 11, 14, 16, 17, 18, 20, 21, 23, 24) THEN 'On Trade'
	WHEN id IN (1, 12, 15, 19, 22) THEN 'Tradi Off.Trade'
	WHEN id IN (4, 25) THEN 'Tier-2 WHS'
	WHEN id IN (13) THEN 'Online sellers'
	ELSE 'OTHERS'
END AS channel_group,
CASE 
	WHEN id IN (2, 9, 10, 11, 14, 16, 17, 18, 20, 21, 23, 24) THEN 1
	WHEN id IN (1, 12, 15, 19, 22) THEN 2
	WHEN id IN (4, 25) THEN 3
	WHEN id IN (13) THEN 4
	ELSE 5
END AS group_order
from Staging.VW_CAT_CHANNEL """).load()

df_sub_channel = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_CAT_SUB_CHANNEL").load()

df_city = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_BI_CITY").load()

df_chain_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_CHAIN_BUSINESS_DASHBOARD").load()


df_business.createOrReplaceTempView('BUSINESS')
df_business_active_data.createOrReplaceTempView('BUSINESS_ACTIVE_DATA')
df_image_level.createOrReplaceTempView('IMAGE_LEVEL')
df_channel.createOrReplaceTempView('CHANNEL')
df_channel_group.createOrReplaceTempView('CHANNEL_GROUP')
df_sub_channel.createOrReplaceTempView('SUB_CHANNEL')
df_city.createOrReplaceTempView('CITY')
df_chain_business.createOrReplaceTempView('CHAIN_BUSINESS')


df_dim_business = spark.sql("""
SELECT b.id, b.name, b.channel_id, c.name as channel_name, c.description as channel_description, b.sub_channel_id, sc.name as sub_channel_name, sc.description as sub_channel_description, cg.channel_group, cg.group_order as channel_group_order, b.image_level_id, i.name as image_level_name, b.city_id, ci.name as city_name, ci.state as state, b.country_id,b.position_id, b.active, b.business_status, b.reporting, created_at, updated_at, first_order_date, last_order_date, cb.business_type, invitation_code, 
CONCAT('R',LEFT(created_at,7)) as registered_month, 
CASE 
WHEN MONTH(created_at) BETWEEN 7 AND 9 THEN CONCAT('RFY',YEAR(created_at)-1999,'-Q1')
WHEN MONTH(created_at) BETWEEN 10 AND 12 THEN CONCAT('RFY',YEAR(created_at)-1999,'-Q2')
WHEN MONTH(created_at) BETWEEN 1 AND 3 THEN CONCAT('RFY',YEAR(created_at)-2000,'-Q3')
WHEN MONTH(created_at) BETWEEN 4 AND 6 THEN CONCAT('RFY',YEAR(created_at)-2000,'-Q4')
END as registered_qtr,
datediff(current_date(),created_at) as days_since_reg,
datediff(current_date(),last_order_date) as days_since_last_order,
acd.active_new, acd.active_start_date, acd.active_end_date
FROM BUSINESS b 
  LEFT JOIN IMAGE_LEVEL i ON b.image_level_id = i.id 
  LEFT JOIN CHANNEL c ON b.channel_id = c.id 
  LEFT JOIN CHANNEL_GROUP cg ON b.channel_id = cg.channel_id 
  LEFT JOIN SUB_CHANNEL sc ON b.sub_channel_id = sc.id 
  LEFT JOIN CITY ci ON b.city_id = ci.id 
  LEFT JOIN 
      (
      SELECT business_id, MAX(business_type_description) as business_type
      FROM CHAIN_BUSINESS
      WHERE business_type is not null
      GROUP BY business_id
      ) cb ON b.id = cb.business_id AND b.country_id = 13      
  LEFT JOIN BUSINESS_ACTIVE_DATA acd ON b.id = acd.id
""")

#display(df_dim_business)

df_dim_business.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Business]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_User: VW_BI_USER join VW_CAT_POSITION

df_dim_user = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""SELECT b.id, b.position_id, c.name as position_name, b.country_id, b.sales_rep as sales_rep_id, email, phone, push_notification, email_notification, sms_notification, whatsapp_notification, created_at, registration_date, last_login_date FROM Staging.VW_BI_USER b LEFT JOIN Staging.VW_CAT_POSITION c ON b.position_id = c.id """).load()

#display(df_dim_user)

df_dim_user.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_User]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Sales_rep: VW_BI_SALES_REP, VW_BI_SALES_REP_BUSINESS, VW_BI_BUSINESS_DASHBOARD, VW_CAT_COUNTRY

df_dim_sales_rep = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT 
id,
email,
chain_id,
name,
last_name,
CASE
    WHEN last_name is not null THEN CONCAT(name,' ',last_name)
    ELSE name
END as sales_rep
FROM Staging.VW_BI_SALES_REP SR
""").load()

df_dim_sales_rep.createOrReplaceTempView('SALES_REP')

# sales_rep language depending on the country of one of its businesses

df_dim_sales_rep_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_SALES_REP_BUSINESS").load()

df_dim_sales_rep_business.createOrReplaceTempView('SALES_REP_BUSINESS')

df_dim_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_BI_BUSINESS_DASHBOARD").load()

df_dim_business.createOrReplaceTempView('BUSINESS')

df_dim_country = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""SELECT id,
CASE
  WHEN name IN ('Colombia','España','Mexico') THEN 'es-ES'
  ELSE 'en-US'
END AS language
FROM Staging.VW_CAT_COUNTRY""").load()

df_dim_country.createOrReplaceTempView('COUNTRY')

df_dim_sales_rep = spark.sql("""
SELECT SR.*, case when c.language is null then 'en-US' else c.language end as language
FROM SALES_REP SR
LEFT JOIN
(
SELECT SRB.sales_rep_id, MIN(SRB.business_id) as business_id
FROM SALES_REP_BUSINESS SRB
GROUP BY SRB.sales_rep_id
) T1
ON SR.id = T1.sales_rep_id
LEFT JOIN BUSINESS B
ON T1.business_id = B.id
LEFT JOIN COUNTRY C
ON B.country_id = C.id
""")

#display(df_dim_sales_rep)

df_dim_sales_rep.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Sales_Rep]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Chain (Wholesaler): VW_CAT_CHAIN_DASHBOARD

df_dim_chain = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user",  conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query", """SELECT id, name, active, country_id, prospects, chain_tier, chain_tier_description, created_at 
FROM Staging.VW_CAT_CHAIN_DASHBOARD """).load()

df_dim_chain.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Chain]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

#display(df_dim_chain)

In [0]:
# Dim_Sales_Rep_Business: VW_BI_SALES_REP_BUSINESS join VW_BI_SALES_REP

df_dim_sales_rep_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_SALES_REP_BUSINESS").load()

df_dim_sales_rep_business.createOrReplaceTempView('SALES_REP_BUSINESS')

df_dim_sales_rep = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_SALES_REP").load()

df_dim_sales_rep.createOrReplaceTempView('SALES_REP')

#### This view was created because of a combination of business-id and chain-id where there are more than 1 sales rep

df_dim_sales_rep_business = spark.sql("""
SELECT SRB.sales_rep_id, SRB.business_id, SR.chain_id, CONCAT(SRB.business_id,'-',SR.chain_id) as business_chain
FROM SALES_REP_BUSINESS SRB
  LEFT JOIN SALES_REP SR ON SRB.sales_rep_id = SR.id 
""")

#display(df_dim_sales_rep_business)

df_dim_sales_rep_business.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Sales_Rep_Business]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_Distributor: VW_CAT_DISTRIBUTION_DASHBOARD
  
df_dim_distribution = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_CAT_DISTRIBUTION_DASHBOARD").load()

#display(df_dim_distribution) 

df_dim_distribution.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Distribution]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_Member_Chain (CPG-Wholesaler): VW_MEMBER_CHAIN_DASHBOARD

df_dim_member_chain = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", 
conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_MEMBER_CHAIN_DASHBOARD").load()

# display(df_dim_member_chain)

df_dim_member_chain.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Member_chain]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_Business_Chain_User: VW_BI_BUSINESS_USER_PARTNER_DASHBOARD (se anula la columna customer_type, viene siempre en 1)

df_dim_business_chain_user = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","SELECT business as business_id, chain as chain_id, [user] as user_id, status, role, CONCAT(business,'-',chain) as business_chain FROM Staging.VW_BI_BUSINESS_USER_PARTNER_DASHBOARD").load()

df_dim_business_chain_user.createOrReplaceTempView('BUSINESS_CHAIN_USER')

#### This view was created because of a combination of business-id and chain-id where there are more than 1 sales rep

# run previous to cmd 12 and cmd 10

df_dim_sales_rep_business.createOrReplaceTempView('SALES_REP_BUSINESS')

df_dim_sales_rep.createOrReplaceTempView('SALES_REP')

df_dim_business_chain_user = spark.sql("""
SELECT BCU.*, SB.sales_rep_id, S.email as sales_rep_email, 
CASE
    WHEN S.last_name is not null THEN CONCAT(S.name,' ',S.last_name)
    ELSE S.name
END as sales_rep
FROM BUSINESS_CHAIN_USER BCU
  LEFT JOIN SALES_REP_BUSINESS SB ON BCU.business_chain = SB.business_chain
  LEFT JOIN SALES_REP S ON SB.sales_rep_id = S.id
""")

#display(df_dim_business_chain_user)

df_dim_business_chain_user.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Business_Chain_User]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Chain_Business: VW_BI_CHAIN_BUSINESS_DASHBOARD join VW_BI_BUSINESS_DASHBOARD

df_dim_chain_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_CHAIN_BUSINESS_DASHBOARD").load()

df_dim_chain_business.createOrReplaceTempView('CHAIN_BUSINESS')

df_channel = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_CAT_CHANNEL").load()

df_channel.createOrReplaceTempView('CHANNEL')


df_dim_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_BI_BUSINESS_DASHBOARD").load()

df_dim_business.createOrReplaceTempView('BUSINESS')


df_dim_chain_business = spark.sql("""
SELECT CB.chain_id, 
CB.business_id, 
CB.business_type, 
CASE 
  WHEN CB.customer_chain_code is null THEN '0' ELSE CB.customer_chain_code
END as business_chain_code, 
CB.business_type_description, 
B.name as business_name,
CONCAT(
CASE 
  WHEN CB.customer_chain_code is null THEN '0' ELSE CB.customer_chain_code
END, '-', B.name)
as business_code_name, 
CONCAT(CB.chain_id,'|',CB.business_id) as chain_business,
b.channel_id as business_channel_id, ch.name as business_channel_name, ch.description as business_channel_description 
FROM CHAIN_BUSINESS CB
LEFT OUTER JOIN BUSINESS B
ON CB.business_id = B.id
LEFT OUTER JOIN CHANNEL CH
ON B.channel_id = CH.id
""")

#display(df_dim_chain_business)

df_dim_chain_business.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Chain_Business]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
#Dim_Country: VW_CAT_COUNTRY

df_dim_country = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""SELECT id, name, active, app_logo, currency_code, nps_min, nps_max, nps_yellow, nps_green,
CASE
  WHEN name = 'Mexico' THEN 'MX'
  WHEN name = 'Singapore' THEN 'SG'
  END AS country_code_GA, 
CASE
  WHEN name IN ('Colombia','España','Mexico') THEN 'es-ES'
  ELSE 'en-US'
END AS language,
CASE
  WHEN name = 'Colombia' THEN 'M'
  ELSE 'K'
END as Unit_Measure_GMV,
CASE
  WHEN name = 'Colombia' THEN 1000000
  ELSE 1000
END as Local_Currency_Divisor  
FROM Staging.VW_CAT_COUNTRY""").load()

df_dim_country.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Country]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

#display(df_dim_country)

In [0]:
# Dim_Promo_Channel: VW_PROMO_CHANNEL

df_dim_promo_channel = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_PROMO_CHANNEL").load()

df_dim_promo_channel.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Promo_Channel]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Promo_SubChannel: VW_PROMO_SUBCHANNEL

df_dim_promo_subchannel = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_PROMO_SUBCHANNEL").load()

df_dim_promo_subchannel.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Promo_SubChannel]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Promo_Image_level: VW_PROMO_IMAGE_LEVEL

df_dim_promo_image_level = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_PROMO_IMAGE_LEVEL").load()

df_dim_promo_image_level.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Promo_Image_Level]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Promo_Business: VW_PROMO_BUSINESS

df_dim_promo_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_PROMO_BUSINESS").load()

df_dim_promo_business.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Promo_Business]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Promo_Business_Type: VW_PROMO_BUSINESS_TYPE

df_dim_promo_business_type = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_PROMO_BUSINESS_TYPE").load()

df_dim_promo_business_type.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Promo_Business_Type]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Promo_by_business: VW_BI_BUSINESS_DASHBOARD, VW_PROMO_BUSINESS, VW_PROMO_CHANNEL, VW_PROMO_SUBCHANNEL

df_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query", """SELECT id, channel_id, sub_channel_id
FROM Staging.VW_BI_BUSINESS_DASHBOARD""").load()

df_promo_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_PROMO_BUSINESS").load()

df_promo_channel = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_PROMO_CHANNEL").load()

df_promo_subchannel = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_PROMO_SUBCHANNEL").load()

df_business.createOrReplaceTempView('BUSINESS') 
df_promo_business.createOrReplaceTempView('PROMO_BUSINESS') 
df_promo_channel.createOrReplaceTempView('PROMO_CHANNEL')
df_promo_subchannel.createOrReplaceTempView('PROMO_SUBCHANNEL')

df_promo_by_business = spark.sql("""
SELECT business_id, 
CASE WHEN channel_id IS NULL THEN 0 ELSE channel_id END as channel_id, 
CASE WHEN sub_channel_id IS NULL THEN 0 ELSE sub_channel_id END as sub_channel_id, 
CASE WHEN product_id IS NULL THEN 0 ELSE product_id END as product_id
FROM
(
SELECT id as business_id, b.channel_id, b.sub_channel_id, pb.product_id
FROM BUSINESS b
INNER JOIN
	PROMO_BUSINESS pb
ON b.id = pb.business_id
UNION
SELECT id as business_id, b.channel_id, sub_channel_id, pc.product_id
FROM BUSINESS b
INNER JOIN
	PROMO_CHANNEL pc
ON b.channel_id = pc.channel_id
UNION
SELECT id as business_id, b.channel_id, sub_channel_id, ps.product_id
FROM BUSINESS b
INNER JOIN
	PROMO_SUBCHANNEL ps
ON b.sub_channel_id= ps.subchannel_id
) T1
""")

# display(df_promo_by_business)

df_promo_by_business.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Promo_by_Business]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_Promo_Product

df_dim_promo_product = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT *, 
  CASE 
    WHEN COUNT(*) OVER (PARTITION BY promo_id) > 1 then 'Mixed Bundle' 
    WHEN quantity > 1 THEN 'Multi Buy' 
    ELSE 'Discount' 
  END AS promo_type,
  ROW_NUMBER() OVER (PARTITION BY promo_id ORDER BY quantity DESC) AS rn
FROM Staging.VW_PROMO_PRODUCT
""").load()

#display(df_dim_promo_product)

df_dim_promo_product.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Promo_Product]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# New Fact_Order: VW_BI_ORDER_DASHBOARD, VW_BI_RATE_VIEW, VW_SALES_REP, VW_SALES_REP_BUSINESS
# Asignar sales rep a ordenes que no lo tienen (con el primer sales rep que aparece para una combinacion business-chain)

df_fact_order = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""SELECT o.id,o.client_order_number,o.chain_id,o.business as business_id,
user_id,u.sales_rep as user_sales_rep_id,o.distributor_id,o.country_id,o.status,
CASE
  WHEN o.status in ('cancelled', 'not_approved') THEN 0
  ELSE 1
END as valid,
original_created_at, o.created_at, o.updated_at, o.observation, o.updated_by, o.customer, o.day_of_preference, o.delivery_cost
FROM Staging.VW_ORDER_DASHBOARD as o 
LEFT JOIN Staging.VW_BI_USER as u
ON o.user_id = u.id
""").load()

df_fact_order.createOrReplaceTempView('FACT_ORDER')

df_fact_rate = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT id_country as country_id, calendar_year, calendar_month, rate, 
id_country + '|' + convert(char(6),(calendar_year * 100 + calendar_month)) as key_rate
FROM Staging.VW_BI_RATE_VIEW
--WHERE rate IS NOT null and rate != 0
""").load()

df_fact_rate.createOrReplaceTempView('RATE')

# Assign sales rep_id based on the first sales_rep that corresponds to the chain-business
df_business_chain_sales_rep=spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT SR.chain_id, SRB.business_id, MIN(SR.id) AS sales_rep_id
FROM Staging.VW_BI_SALES_REP SR
INNER JOIN Staging.VW_BI_SALES_REP_BUSINESS SRB ON SR.id = SRB.sales_rep_id
GROUP BY SR.chain_id, SRB.business_id
""").load()

df_business_chain_sales_rep.createOrReplaceTempView('SALESREP_BUSINESS_CHAIN')

#display(df_business_chain_sales_rep)

df_query_lines = spark.read.format("jdbc").option("url", conf['jdbcUrl_origin']).option("driver", conf['jdbcDriver_origin']).option("user", conf['jdbcUsername_origin']).option("password", conf['jdbcPassword_origin']).option("query","""
SELECT [order] as order_id, 
COUNT(*) as total_lines,
SUM(CASE WHEN status = 'cancelled' THEN 1 ELSE 0 END) as total_cancelled_lines,
SUM(CASE WHEN status in ('cancelled', 'not_approved', 'returned') THEN 0 ELSE current_quantity * current_unit_price - discount END) as total_amount,
SUM(CASE WHEN status = 'cancelled' THEN current_quantity * current_unit_price - discount ELSE 0 END) as cancelled_amount
FROM VW_ORDER_LINE_DASHBOARD 
GROUP BY [order]
""").load()

df_query_lines.createOrReplaceTempView('QUERY_LINES')

df_fact_order = spark.sql("""
SELECT o.*, concat(o.country_id,'|',year(o.created_at)*100 + month(o.created_at)) as key_rate, r.rate, 
CASE WHEN o.user_sales_rep_id IS NOT NULL THEN o.user_sales_rep_id ELSE sbc.sales_rep_id END as sales_rep_id, 
concat(o.business_id,'-',o.chain_id) as business_chain, ol.total_lines, round(ol.total_amount,2) as total_amount,
CASE 
  WHEN ol.total_cancelled_lines > 0 AND ol.total_cancelled_lines <> ol.total_lines THEN 'Partial'
  WHEN ol.total_cancelled_lines > 0 AND ol.total_cancelled_lines = ol.total_lines THEN 'Total'
END as cancellation_status,
ol.total_cancelled_lines, ol.cancelled_amount,
lag(o.created_at) over (partition by o.chain_id,o.business_id order by o.id) as previous_chain_business_order_date,
datediff(o.created_at,lag(o.created_at) over (partition by o.chain_id,o.business_id order by o.id)) as days_from_last_order
FROM FACT_ORDER o 
LEFT OUTER JOIN RATE r ON concat(o.country_id,'|',year(o.created_at)*100 + month(o.created_at)) = r.key_rate 
LEFT OUTER JOIN SALESREP_BUSINESS_CHAIN sbc ON o.business_id = sbc.business_id AND o.chain_id = sbc.chain_id
LEFT OUTER JOIN QUERY_LINES ol ON o.id = ol.order_id
""")


#display(df_fact_order)

df_fact_order.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_Order]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# New Fact_order_line table (original rows from view VW_ORDER_LINE_DASHBOARD, that's showing promo products + 
# transformed lines of promo_products by the products included in the promo

#
# Original_order_line: VW_ORDER_LINE_DASHBOARD
#

df_original_order_line = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT ol.line_number, CONVERT(NVARCHAR(MAX),ol.id) as id_surrogate,
ol.product_id, ol.current_quantity, ol.current_unit_price, ol.delivery_date, ol.invoice_number, ol.[status], 
CASE 
WHEN o.country_id = '13' and YEAR(ol.created_at) <= 2022 THEN 1.07
WHEN o.country_id = '13' and YEAR(ol.created_at) > 2022 THEN 1.08 
ELSE 1 
END AS excise,
CASE
WHEN ol.[status] in ('cancelled', 'not_approved', 'returned') THEN 0
ELSE 1
END as valid,
CASE
WHEN ol.[status] in ('cancelled', 'not_approved', 'returned') THEN 0
ELSE ol.current_quantity
END as current_quantity_final,
CASE
WHEN ol.[status] in ('cancelled', 'not_approved', 'returned') THEN 0
ELSE ol.current_quantity * ol.current_unit_price - ol.discount
END as line_amount,
CASE WHEN ol.[status] = 'cancelled' THEN ol.current_quantity ELSE 0 END as cancelled_quantity,
CASE WHEN ol.[status] = 'cancelled' THEN ol.current_quantity * ol.current_unit_price - ol.discount ELSE 0 END as cancelled_amount,
CASE WHEN ol.[status] = 'cancelled' THEN cancellation_id ELSE NULL END as cancellation_id, 
ol.updated_at, ol.updated_by, ol.[order], o.country_id, ol.uom, ol.id, ol.observation, ol.created_at,
ol.payment_date, ol.invoice_date, ol.discount, ol.taxes, ol.presentation, ol.presentation_quantity, ol.cost_per_unit,
0 as line_transformed,
CASE WHEN Promo.promo_id IS NULL THEN 'Normal Product' ELSE 'Promo Product' END as product_flag,
null as promo_row,
null as promo_id,
null as promo_product_id,
CONVERT(NVARCHAR(MAX),null) as promo_type
FROM Staging.VW_ORDER_LINE_DASHBOARD as ol 
JOIN Staging.VW_ORDER_DASHBOARD AS o ON ol.[order]=o.id
LEFT JOIN (SELECT DISTINCT promo_id FROM Staging.VW_PROMO_PRODUCT) Promo ON ol.product_id = Promo.promo_id
""").load()

df_original_order_line.createOrReplaceTempView('ORIGINAL_FACT_ORDER_LINE')

#display(df_original_order_line)

#
# End original_order_line: VW_ORDER_LINE_DASHBOARD
#

#
# Transformed_order_line: VW_ORDER_LINE_DASHBOARD join VW_PROMO_PRODUCT 
#

df_dim_promo_product = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT *, 
  CASE 
    WHEN COUNT(*) OVER (PARTITION BY promo_id) > 1 then 'Mixed Bundle' 
    WHEN quantity > 1 THEN 'Multi Buy' 
    ELSE 'Discount' 
  END AS promo_type,
 	p.volume as promo_product_volume,
	p.volume_measure as promo_product_volume_measure
FROM Staging.VW_PROMO_PRODUCT PP
  INNER JOIN Staging.VW_CAT_PRODUCT P ON pp.product_id = p.id 
""").load()

df_dim_promo_product.createOrReplaceTempView('PROMO_PRODUCT')
 
df_transformed_order_line = spark.sql("""
SELECT 
	ol.line_number,
	concat(ol.id,'/',rn) as id_surrogate,
	dp.product_id as product_id,
  dp.quantity * current_quantity as current_quantity,
  dp.quantity * current_quantity_final as current_quantity_final,
	case 
		when dp.quantity = 0 then 0
    when rn = 1 then ol.current_unit_price / dp.quantity 
    else null 
  end as current_unit_price, 
	delivery_date,
	invoice_number,
	status,
	valid,
 	case when rn = 1 then ol.line_amount else null end as line_amount,
 	case when rn = 1 then ol.cancelled_amount else null end as cancelled_amount,
	dp.quantity * ol.cancelled_quantity as cancelled_quantity,
	cancellation_id,
	updated_at,
	updated_by,
	order,
	ol.country_id,
 	case when rn = 1 then ol.uom else null end as uom,
  ol.id,
	observation, 
	created_at,
	payment_date,
	invoice_date,
 	case when rn = 1 then ol.discount else null end as discount, 
 	case when rn = 1 then ol.taxes else null end as taxes, 
  ol.presentation,
  ol.presentation_quantity, 
 	case 
		when dp.quantity = 0 then 0
		when rn = 1 then cost_per_unit / dp.quantity 
    else null
  end as cost_per_unit, 
	excise,
	-- Below data of products in promos
	rn as promo_row,
	dp.promo_id,
	dp.promo_type,
	dp.product_id as promo_product_id,
	1 as line_transformed,
  'Promo Normal Product' as product_flag
FROM ORIGINAL_FACT_ORDER_LINE ol
INNER JOIN
(
SELECT *,
    ROW_NUMBER() OVER (PARTITION BY promo_id ORDER BY quantity DESC) AS rn
    FROM PROMO_PRODUCT
) dp ON ol.product_id = dp.promo_id
""")

df_transformed_order_line.createOrReplaceTempView('TRANSFORMED_ORDER_LINE')

#display(df_transformed_order_line)

#
# End Transformed_order_line: VW_ORDER_LINE_DASHBOARD join VW_PROMO_PRODUCT 
#

#
# fact_order_line_v2: append original + transformed order lines
#

df_fact_order_line_v2 = spark.sql("""
select line_number, id_surrogate, product_id, current_quantity, current_unit_price, delivery_date, invoice_number, status, excise,
valid, current_quantity_final, line_amount, cancelled_quantity, cancelled_amount, cancellation_id, updated_at, updated_by,
order, country_id, uom, id, observation, created_at, payment_date, invoice_date, discount, taxes, presentation, presentation_quantity,
cost_per_unit, line_transformed, product_flag, promo_row, promo_id, promo_product_id, promo_type
from TRANSFORMED_ORDER_LINE
union all
select line_number, id_surrogate, product_id, current_quantity, current_unit_price, delivery_date, invoice_number, status, excise,
valid, current_quantity_final, line_amount, cancelled_quantity, cancelled_amount, cancellation_id, updated_at, updated_by,
order, country_id, uom, id, observation, created_at, payment_date, invoice_date, discount, taxes, presentation, presentation_quantity,
cost_per_unit, line_transformed, product_flag, promo_row, promo_id, promo_product_id, promo_type
from ORIGINAL_FACT_ORDER_LINE
""")

df_fact_order_line_v2.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_Order_Line]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


#
# End fact_order_line: append original + transformed order lines
#

In [0]:
# Dim_Hours_Order

df_dim_Hours_Order = spark.read.format("jdbc").option("url", conf['jdbcUrl_origin']).option("driver", conf['jdbcDriver_origin']).option("user", conf['jdbcUsername_origin']).option("password", conf['jdbcPassword_origin']).option("query","""
SELECT 0 as Hourid, '00 to 01' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 1 as Hourid, '01 to 02' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 2 as Hourid, '02 to 03' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 3 as Hourid, '03 to 04' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 4 as Hourid, '04 to 05' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 5 as Hourid, '05 to 06' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 6 as Hourid, '06 to 07' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 7 as Hourid, '07 to 08' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 8 as Hourid, '08 to 09' as HourRange, 'MORNING' as 'HourDescription', 2 as 'HourDescriptionOrder' UNION
SELECT 9 as Hourid, '09 to 10' as HourRange, 'MORNING' as 'HourDescription', 2 as 'HourDescriptionOrder' UNION
SELECT 10 as Hourid, '10 to 11' as HourRange, 'MORNING' as 'HourDescription', 2 as 'HourDescriptionOrder' UNION
SELECT 11 as Hourid, '11 to 12' as HourRange, 'MORNING' as 'HourDescription', 2 as 'HourDescriptionOrder' UNION
SELECT 12 as Hourid, '12 to 13' as HourRange, 'MORNING' as 'HourDescription', 2 as 'HourDescriptionOrder' UNION
SELECT 13 as Hourid, '13 to 14' as HourRange, 'AFTERNOON' as 'HourDescription', 3 as 'HourDescriptionOrder' UNION
SELECT 14 as Hourid, '14 to 15' as HourRange, 'AFTERNOON' as 'HourDescription', 3 as 'HourDescriptionOrder' UNION
SELECT 15 as Hourid, '15 to 16' as HourRange, 'AFTERNOON' as 'HourDescription', 3 as 'HourDescriptionOrder' UNION
SELECT 16 as Hourid, '16 to 17' as HourRange, 'AFTERNOON' as 'HourDescription', 3 as 'HourDescriptionOrder' UNION
SELECT 17 as Hourid, '17 to 18' as HourRange, 'AFTERNOON' as 'HourDescription', 3 as 'HourDescriptionOrder' UNION
SELECT 18 as Hourid, '18 to 19' as HourRange, 'AFTERNOON' as 'HourDescription', 3 as 'HourDescriptionOrder' UNION
SELECT 19 as Hourid, '19 to 20' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 20 as Hourid, '20 to 21' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 21 as Hourid, '21 to 22' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 22 as Hourid, '22 to 23' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' UNION
SELECT 23 as Hourid, '23 to 24' as HourRange, 'OOO' as 'HourDescription', 1 as 'HourDescriptionOrder' 
""").load()

#display(df_dim_HoursOrder)

df_dim_Hours_Order.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Hours_Order]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Fact_NPS_Result: VW_NPS_RESULT_DASHBOARD 
 
df_dim_chain = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user",  conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query", """SELECT id, name FROM Staging.VW_CAT_CHAIN_DASHBOARD """).load()
 
df_dim_chain.createOrReplaceTempView('WHS')
 
# display(df_dim_chain)
 
df_nps = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""SELECT survey_start_date, survey_end_date, chain_id, business_id, score, 
CASE
  WHEN score <= 6 THEN 'Detractor'
  WHEN score <= 8 THEN 'Passive'
  ELSE 'Promoter'
END AS category,
date, country_id, order_id, comment 
FROM Staging.VW_NPS_RESULT_DASHBOARD
WHERE score > 0""").load()

df_nps.createOrReplaceTempView('NPS')

df_fact_nps_result = spark.sql("""
SELECT survey_start_date, survey_end_date, n.chain_id, business_id, score, category, date, country_id, order_id, comment, w.name as WHS_name
FROM NPS n
  LEFT OUTER JOIN WHS w
  ON n.chain_id = w.id
""")
 
# display(df_fact_nps_result)
 
df_fact_nps_result.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_NPS_Result]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Fact_Rate: VW_BI_RATE_VIEW 

df_fact_rate = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT id_country as country_id, calendar_year, calendar_month, rate, 
id_country + '|' + convert(char(6),(calendar_year * 100 + calendar_month)) as key_rate
FROM Staging.VW_BI_RATE_VIEW
--WHERE rate IS NOT null and rate != 0
""").load()

#display(df_fact_rate)

df_fact_rate.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_Rate]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Fact_Target: VW_BI_TARGET_VIEW

df_fact_target = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT id_country as country_id, calendar_year, calendar_month, target, 
id_country + '|' + convert(char(6),(calendar_year * 100 + calendar_month)) as key_rate
FROM Staging.VW_BI_TARGET_VIEW""").load()

df_fact_target.createOrReplaceTempView('TARGET')

df_fact_rate = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT id_country as country_id, calendar_year, calendar_month, rate, 
id_country + '|' + convert(char(6),(calendar_year * 100 + calendar_month)) as key_rate
FROM Staging.VW_BI_RATE_VIEW
""").load()

df_fact_rate.createOrReplaceTempView('RATE')

df_fact_target = spark.sql("""
SELECT t.country_id, t.calendar_year, t.calendar_month, t.target, t.key_rate,
CASE WHEN t.calendar_month > 6 THEN t.calendar_month - 6
ELSE t.calendar_month + 6
END as order_month,
cast(concat(t.calendar_year,'-',t.calendar_month,'-',1) as date) as Fecha,
r.rate,
t.target * r.rate as target_euros,
t.calendar_year * 100 + t.calendar_month as Year_Month,
CASE
WHEN t.country_id = '13' and t.calendar_year <= 2022 THEN 1.07
WHEN t.country_id = '13' and t.calendar_year > 2022 THEN 1.08
ELSE 1
END as excise
FROM TARGET t
  LEFT OUTER JOIN RATE r
  ON t.key_rate = r.key_rate
""")

#display(df_fact_target)

df_fact_target.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_Target]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Fact_Notification_Tracking: join views VW_NOTIFICATION_DASHBOARD, VW_CART_DASHBOARD, VW_CART_LINE_DASHBOARD, VW_ORDER_DASHBOARD, VW_ORDER_LINE_DASHBOARD, VW_CAT_PRODUCT

df_notif = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
    SELECT country, business_id as business, chain, [user_id], promotion_id, pro.name as product, configuration_id, noti.created_at, sent_at, opened_at
    FROM Staging.VW_NOTIFICATION_DASHBOARD noti
    LEFT OUTER JOIN
        Staging.VW_CAT_PRODUCT pro
    ON noti.promotion_id = pro.id
""").load()

df_cart = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT id, country_id, [user_id], business, chain_id, created_at as add_to_cart_at 
FROM Staging.VW_CART_DASHBOARD
""").load()

df_cart_line = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT cart, product
FROM Staging.VW_CART_LINE_DASHBOARD
WHERE product IS NOT null
""").load()

df_order = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT id, country_id, [user_id], business, chain_id, created_at 
FROM Staging.VW_ORDER_DASHBOARD
""").load()

df_order_line = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","""
SELECT product_id,[order] 
FROM Staging.VW_ORDER_LINE_DASHBOARD
""").load()

df_notif.createOrReplaceTempView('NOTIF')
df_cart.createOrReplaceTempView('CART')
df_cart_line.createOrReplaceTempView('CART_LINE')
df_order.createOrReplaceTempView('ORDERS')
df_order_line.createOrReplaceTempView('ORDER_LINE')

df_fact_not_track = spark.sql("""
SELECT country, business, chain, user_id, promotion_id, product, configuration_id, created_at, sent_at, opened_at, add_to_cart_at, 
  CASE 
    WHEN bought_at > add_to_cart_at then bought_at 
    ELSE null 
  END as bought_at 
FROM
(
  SELECT T1.country, T1.business, T1.chain, T1.user_id, T1.promotion_id, T1.product, T1.configuration_id, T1.created_at, T1.sent_at, T1.opened_at, 
  CASE 
    WHEN T2.add_to_cart_at > T1.opened_at THEN T2.add_to_cart_at 
    ELSE null 
  END as add_to_cart_at, T3.last_date_order as bought_at
  FROM
    ( SELECT * FROM NOTIF ) as T1
  LEFT OUTER JOIN
    (
    /* Busco ultima fecha de cada producto para cada user+business+chain en el carrito */
    SELECT country_id, user_id, business, chain_id, product, MAX(add_to_Cart_at) as add_to_Cart_at
    FROM 
      ( SELECT * FROM CART ) as aux1
    INNER JOIN
      ( SELECT * FROM CART_LINE ) as aux2
    ON aux1.id = aux2.cart
    GROUP BY country_id, user_id, business, chain_id, product
    ) as T2
  ON T1.country = T2.country_id
  AND T1.chain = T2.chain_id
  AND T1.business = T2.business
  AND T1.user_id = T2.user_id
  AND T1.promotion_id = T2.product
  LEFT OUTER JOIN
  (
      /* Busco ultima fecha de compra de cada producto para cada user+business+chain */
    SELECT country_id, user_id, business, chain_id, product_id, MAX(aux3.created_at) as last_Date_order
    FROM
      ( SELECT * FROM ORDERS ) as aux3
    INNER JOIN
      ( SELECT * FROM ORDER_LINE ) as aux4
    ON aux3.id = aux4.order
    GROUP BY country_id, user_id, business, chain_id, product_id
  ) as T3
  ON T1.country = T3.country_id AND T1.chain = T3.chain_id AND T1.business = T3.business AND T1.user_id = T3.user_id AND T1.promotion_id = T3.product_id
) as Tmp_table 
""")

#display(df_fact_not_track)

df_fact_not_track.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_Notification_Tracking]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Fact_analytics_search: VW_ANALYTICS_SEARCH_DASHBOARD

df_fact_analytics_search = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_ANALYTICS_SEARCH_DASHBOARD").load()

#df_fact_analytics_search=df_fact_analytics_search.withColumn("created_at",unix_timestamp(df_fact_analytics_search["created_at"], "yyyy-MM-dd HH:mm:ss").cast("timestamp"))

df_fact_analytics_search.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_Analytics_Search]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

#display(df_fact_analytics_search)

In [0]:
# Google Analytics

# 29/05/24 This notebook is disabled because it produces the following error:
# net.snowflake.client.jdbc.SnowflakeSQLException: User access disabled. Contact your local system administrator.

# options = {
#   "sfUrl": dbutils.secrets.get('key-vault-company-secrets', 'sf-googleanalytics-url'),
#   "sfUser": dbutils.secrets.get('key-vault-company-secrets', 'sf-googleanalytics-user'),
#   "sfPassword": dbutils.secrets.get('key-vault-company-secrets', 'sf-googleanalytics-password'),
#   "sfDatabase": dbutils.secrets.get('key-vault-company-secrets', 'sf-googleanalytics-db'),
#   "sfSchema": dbutils.secrets.get('key-vault-company-secrets', 'sf-googleanalytics-schema'),
#   "sfWarehouse": dbutils.secrets.get('key-vault-company-secrets', 'sf-googleanalytics-warehouse')
# }

# df_Fact_GA = spark.read \
#   .format("snowflake") \
#   .options(**options) \
#   .option("query",  """
#   SELECT VISITSTARTTIME, TOTALS_TIMEONSITE, DEVICE_OPERATINGSYSTEM, DEVICE_DEVICECATEGORY, GEONETWORK_COUNTRY, GEONETWORK_CITY, 
#   HITS_PAGE_HOSTNAME, FULLVISITORID, 1 as Enabled  
#   FROM PRLAT_PROD_DB.SCH_PRLAT_CONSUMER_GOOGLE_ANALYTICS.GOOGLE_ANALYTICS_RAW_DATA 
#   WHERE WEB_PROPERTY_ID = 'UA-99306742-7' 
#     AND 
#     (HITS_PAGE_HOSTNAME = 'app.e-anaquel.com.mx' or HITS_PAGE_HOSTNAME = 'app.company.com.sg' 
#      or HITS_PAGE_HOSTNAME = 'www.e-anaquel.com.mx')""") \
#   .load()

# df_Fact_GA.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_GA]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Dim_Sys_Users: VW_BI_CAT_SYS_USER

df_vw_sys_user = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_CAT_SYS_USER").load()

df_vw_sys_user.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.Dim_Sys_Users").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_Sys_Users_Chain: join view VW_SYS_USER_CHAIN_ACCESS_DASHBOARD, VW_BI_CAT_SYS_USER

df_vw_sys_user_chain = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_SYS_USER_CHAIN_ACCESS_DASHBOARD").load()

df_vw_sys_user_chain.createOrReplaceTempView('SYSUSERS_CHAIN')

df_vw_sys_user = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_CAT_SYS_USER").load()

df_vw_sys_user.createOrReplaceTempView('SYSUSERS')

df_sys_user_chain = spark.sql("""
SELECT C.user_id as sys_user, C.chain, U.email AS sys_user_email, C.country
FROM SYSUSERS_CHAIN C 
INNER JOIN SYSUSERS U
ON C.user_id = U.user_id
""")

#display(df_sys_urser_chain)

df_sys_user_chain.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.Dim_Sys_Users_Chain").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()



In [0]:
# Dim_Sys_Users_Member: join views VW_SYS_USER_MEMBER, VW_BI_CAT_SYS_USER, VW_CAT_MEMBER

df_vw_sys_user_member = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_SYS_USER_MEMBER").load()

df_vw_sys_user_member.createOrReplaceTempView('SYSUSERS_MEMBER')

df_vw_sys_user = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_CAT_SYS_USER").load()

df_vw_sys_user.createOrReplaceTempView('SYSUSERS')

df_vw_member = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_CAT_MEMBER").load()

df_vw_member.createOrReplaceTempView('MEMBERS')

df_sys_user_member = spark.sql("""
SELECT M.sys_user, M.member, U.email AS sys_user_email, U.country_id
FROM SYSUSERS_MEMBER M 
INNER JOIN SYSUSERS U
ON M.sys_user = U.user_id
UNION
-- add users with role ADMIN (all CPGs)
SELECT U.user_id AS sys_user, M.id AS member, U.email, M.country_id
FROM SYSUSERS U
CROSS JOIN MEMBERS M
WHERE role IN ('ADMIN','GLOBAL_REPORT') and email like '%@%'
UNION
-- add users with role COUNTRY_ADMIN (all CPGs del mismo pais que el usuario)
SELECT U.user_id AS sys_user, M.id AS member, U.email, U.country_id 
FROM SYSUSERS U
INNER JOIN MEMBERS M ON U.country_id = M.country_id
WHERE role IN ('COUNTRY_ADMIN','COUNTRY_MANAGER_ADMIN') and email like '%@%'
""")

#display(df_sys_urser_member)

df_sys_user_member.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.Dim_Sys_Users_Member").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_Sys_Users_ProductCategory: join views df_sys_user_member (previous dataframe), VW_CAT_BRAND, VW_CAT_PRODUCT

df_sys_user_member.createOrReplaceTempView('USER_MEMBERS')

df_dim_brand = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_CAT_BRAND").load()

df_dim_brand.createOrReplaceTempView('BRANDS')

df_dim_product = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_CAT_PRODUCT").load()

df_dim_product.createOrReplaceTempView('PRODUCTS')

df_sysuser_productcategory = spark.sql("""
SELECT DISTINCT UM.sys_user_email, UM.sys_user, UM.country_id, P.product_category_id
FROM USER_MEMBERS UM
INNER JOIN BRANDS B ON UM.member = B.member
INNER JOIN PRODUCTS P ON B.id = P.brand_id
""")

#display(df_sysuser_productcategory)

df_sysuser_productcategory.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_SysUser_ProductCategory]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_ProductCategory: the data of this table is already in the product dimension, an independent table is created to solve the RLS problem in CPG

df_product_category = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("query","SELECT id, name FROM Staging.VW_BI_CAT_PRODUCT_CATEGORY").load()

df_product_category.createOrReplaceTempView('PRODUCT_CATEGORY')

#display(df_product_category)

df_product_category.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Product_Category]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Stock Level

#df_stock_level = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_STOCK_LEVEL").load()

#df_stock_level.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_Stock]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


In [0]:
# Stock Level

# This code will retry the transaction up to three times if a deadlock occurs, with a delay of 5 seconds between each attempt. If the transaction is successful, it will exit the loop. If the maximum number of retries is reached, it will raise the exception.

max_retries = 3
retry_delay = 5  # seconds

for attempt in range(max_retries):
    try:
        df_stock_level = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable", "Staging.VW_STOCK_LEVEL").load()

        df_stock_level.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fact_Stock]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()
        break  # Exit the loop if the transaction is successful
    except Exception as e:
        if "deadlock" in str(e):
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
            else:
                raise
        else:
            raise

In [0]:
# Dim_Cancellation_Reason: VW_CAT_ CANCELLATION_REASON

df_cancellation_reason = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", 
conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_CAT_CANCELLATION_REASON").load()

#display(df_cancellation_reason)

df_cancellation_reason.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Cancellation_Reason]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Dim_Member_Business: join views VW_Member_Chain and VW_Chain_Business"

df_member_chain = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_MEMBER_CHAIN_DASHBOARD").load()

df_member_chain.createOrReplaceTempView('MEMBER_CHAIN')

#display(df_member_chain)

df_chain_business = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_BI_CHAIN_BUSINESS_DASHBOARD").load()

df_chain_business.createOrReplaceTempView('CHAIN_BUSINESS')

#display(df_chain_business)

df_member_business = spark.sql("""
SELECT member, business_id
FROM MEMBER_CHAIN MB
INNER JOIN CHAIN_BUSINESS CB ON MB.chain = CB.chain_id
GROUP BY member, business_id
""")

#display(df_member_business)

df_member_business.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Dim_Member_Business]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

In [0]:
# Fast_order_tracking, Fast_order_tracking_found_line, Fast_order_tracking_not_found_line

df_fast_order_tracking_found_line = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_FAST_ORDER_TRACKING_FOUND_LINE").load()

df_fast_order_tracking_found_line.createOrReplaceTempView('FAST_ORDER_FOUND_LINE')

df_Fast_Order_Tracking = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_FAST_ORDER_TRACKING").load()

df_Fast_Order_Tracking.createOrReplaceTempView('FAST_ORDER_TRACKING')

df_Fact_Order_Line = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","companyDummy.Fact_Order_Line").load()

df_Fact_Order_Line.createOrReplaceTempView('FACT_ORDER_LINE')

df_Fact_Order = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","companyDummy.Fact_Order").load()

df_Fact_Order.createOrReplaceTempView('FACT_ORDER')


### Fast_Order_Tracking_Found_line

df_view_found_line = spark.sql("""
SELECT fa.*, product_purchase
from FAST_ORDER_FOUND_LINE fa
inner join 
	(
	SELECT fa.productId, fa.fastOrderTrackingId,
		 ROW_NUMBER() OVER (PARTITION BY fa.productId, fa.fastOrderTrackingId ORDER BY fa.fastOrderTrackingId,fa.productId) AS row_num,
		 CASE WHEN lo.product_id is null THEN 'NOT PURCHASE' ELSE 'PURCHASE' END as product_purchase
	FROM FAST_ORDER_FOUND_LINE fa
	INNER JOIN FAST_ORDER_TRACKING ft
		ON fa.fastOrderTrackingId = ft.id
	LEFT JOIN FACT_ORDER_LINE lo
		ON  ft.orderId = lo.order
		AND lo.product_id = fa.productId
		AND lo.product_flag IN ('Normal Product', 'Promo Product')
		AND lo.valid = 1
	) a
    on fa.fastOrderTrackingId = a.fastOrderTrackingId
    and fa.productId = a.productId
	  and row_num = 1
""")

df_view_found_line.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fast_Order_Tracking_Found_line]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


### Fast_order_tracking

df_view_fast_order = spark.sql("""
SELECT ft.*,
    CASE WHEN  ft.orderId is null THEN 'NO PURCHASE'
        ELSE 'PURCHASE' END as RecommendationFunnel1,
    CASE WHEN MORE_ALGOLIA is null then NULL
         WHEN MORE_ALGOLIA = 0 AND MORE_ORDERS = 0 THEN 'PERFECT_MATCH'
         WHEN MORE_ALGOLIA = 0 AND MORE_ORDERS > 0 THEN 'ADD'
         WHEN MORE_ORDERS = 0 AND MORE_ALGOLIA > 0 THEN 'DROP'
    ELSE 'PARTIAL_MATCH' END as RecommendationFunnel2,
fo.created_at AS order_created_at,
fo.valid AS order_valid
FROM FAST_ORDER_TRACKING ft
LEFT JOIN FACT_ORDER fo ON ft.orderId = fo.id
LEFT JOIN 
	(
    SELECT orderId, COUNT(orderId) AS ORDERS,
        SUM(CASE WHEN (productId = product_id)  AND ( product_id is not null) THEN 1 ELSE 0 END) as MATCH,
        SUM(CASE WHEN (product_id is null)  AND ( productId is not null) THEN 1 ELSE 0 END) as MORE_ALGOLIA,
        SUM(CASE WHEN (productId is null)  AND ( product_id is not null) THEN 1 ELSE 0 END) as MORE_ORDERS
    FROM 
	(
		SELECT orderId,  product_id, productId
		FROM 
		(
			SELECT ft.orderId,  lo.product_id, fa.productId
			FROM FAST_ORDER_TRACKING ft
			INNER JOIN FAST_ORDER_FOUND_LINE fa
				ON fa.fastOrderTrackingId = ft.id
			LEFT JOIN FACT_ORDER_LINE lo
				ON  ft.orderId = lo.order
				AND lo.product_id = fa.productId
				AND lo.product_flag IN ('Normal Product', 'Promo Product')
				AND lo.valid = 1
		) more_orders_explode
		UNION ALL
		SELECT orderId,  product_id, productId
		FROM 
		(
			SELECT ft.orderId,  lo.product_id, fa.productId
			FROM FAST_ORDER_TRACKING ft
			INNER JOIN FACT_ORDER_LINE lo
				ON  ft.orderId = lo.order
			LEFT JOIN FAST_ORDER_FOUND_LINE fa
				ON fa.fastOrderTrackingId = ft.id
				AND lo.product_id = fa.productId 
				AND lo.product_flag IN ('Normal Product', 'Promo Product')
				AND lo.valid = 1
			WHERE fa.productId  is NULL 
		) more_algolia_explode
	) Union_cases
    GROUP BY orderId
	) l
    ON l.orderId = ft.orderId
""")

#display(df_view_fast_order)

df_view_fast_order.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fast_Order_Tracking]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()


### Fast_order_tracking_not_found_line

df_fast_order_tracking_not_found_line = spark.read.format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).option("dbtable","Staging.VW_FAST_ORDER_TRACKING_NOT_FOUND_LINE").load()

df_fast_order_tracking_not_found_line.write.mode("overwrite").format("jdbc").option("url", conf['jdbcUrl_destination']).option("driver", conf['jdbcDriver_destination']).option("dbtable", "companyDummy.[Fast_Order_Tracking_Not_Found_line]").option("user", conf['jdbcUsername_destination']).option("password", conf['jdbcPassword_destination']).save()

